# Importando as bibliotecas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pymannkendall import original_test
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from statsmodels.tsa.seasonal import seasonal_decompose
from tqdm.auto import tqdm

from db import connection_db as conndb
from db import filters
from utils.estracao_interacao import ExtracaoInteracao

tqdm.pandas()

# Conectando ao banco de dados do MongoDB

Função para conectar ao MongoDB

In [ ]:
def conectar_mongodb(uri, db_name, collection_name):
    mongo_connection = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=collection_name)
    mongo_connection.connect()
    return mongo_connection.collection

In [ ]:
# Definindo os parâmetros de conexão
uri = 'mongodb://localhost:27017/'
db_name = 'dadosVivamente'
collection_name = 'dadosSemFiltros'
collection_name_filtered = 'posts7anos1anos'

collection = conectar_mongodb(uri, db_name, collection_name)
collection_filtrada = conectar_mongodb(uri, db_name, collection_name_filtered)

# Aplicando pipeline para preparação dos dados

Função para aplicar os pipelines de filtro

In [ ]:
def aplicar_pipelines(collection, collection_filtrada):
    collection_filters = filters.CollectionFilters(collection)
    collection_filters_likes = filters.CollectionFilters(collection_filtrada)

    # Aplicar pipelines
    collection_filters.apply_pipeline1('dadosComFiltrosIniciais')
    collection_filters.apply_pipeline2(7, 1, 'posts7anos1anos')
    collection_filters.apply_pipeline3('postsComBDIAndInfos')
    collection_filters.apply_pipeline4('posts')
    collection_filters_likes.apply_pipeline6('likes')

    return collection_filters.collection, collection_filters_likes.collection

In [ ]:
# Aplicando os pipelinesc
collection, collection_filtrada = aplicar_pipelines(collection, collection_filtrada)

# Carregando os dados em um dataframe

In [ ]:
documentos = collection.find()
df_original = pd.DataFrame(list(documentos))

In [ ]:
# Visualizar as primeiras linhas do dataframe
df_original.head()

In [ ]:
documentos_likes = collection_filtrada.find()
df_original_likes = pd.DataFrame(list(documentos_likes))

In [ ]:
# Visualizar as primeiras linhas do dataframe
df_original_likes.head()

In [ ]:
# Transformando a coluna sexo em binária
df_original['sexo'] = df_original['sexo'].map({'F': 0, 'M': 1})

# Removendo colunas desnecessárias
df_original.drop(columns=['_id', 'diaDaSemana'], inplace=True)

# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima', 'critica', 'suicida', 'choro',
    'agitacao', 'interesse', 'indecisao', 'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]


# Função para preencher valores nulos e converter o tipo de dado
def preencher_e_converter(df, colunas, valor_preenchimento=0, tipo_dados='int64'):
    df[colunas] = df[colunas].fillna(valor_preenchimento)
    df[colunas] = df[colunas].astype(tipo_dados)
    return df

In [ ]:
df_original = preencher_e_converter(df_original, colunas_para_converter)

In [ ]:
df_original.head()

In [ ]:
# Função para manipular datas e filtrar posts
def manipular_e_filtrar_posts(df):
    df['data'] = df['postCreatedTime'].dt.date
    df['data'] = pd.to_datetime(df['data'])
    df['mes'] = df['data'].dt.to_period('M')
    df['semana'] = df['data'].dt.to_period('W')

    df_data = df[df['data'].dt.year == 2017]
    df_data = df_data[~((df_data['data'].dt.month == 12) & (df_data['data'].dt.year == 2017))]
    df_data = df_data[df_data['data'].dt.month >= 5]
    df_data = df_data[df_data['suicida'] == 3]

    # Remover posts vazios
    df_data = df_data.dropna(subset=['postMessage', 'postStory'], how='all')
    df_data = df_data[~((df_data['postMessage'].str.strip() == '') & (df_data['postStory'].str.strip() == ''))]

    return df_data

In [ ]:
df_posts = manipular_e_filtrar_posts(df_original)
df_posts.head()

In [ ]:
# Função para manipular datas e filtrar posts
def manipular_e_filtrar_likes(df):
    df['data'] = df['likeCreatedTime'].dt.date
    df['data'] = pd.to_datetime(df['data'])
    df['mes'] = df['data'].dt.to_period('M')
    df['semana'] = df['data'].dt.to_period('W')

    df_data = df[df['data'].dt.year == 2017]
    df_data = df_data[~((df_data['data'].dt.month == 12) & (df_data['data'].dt.year == 2017))]
    df_data = df_data[df_data['data'].dt.month >= 5]

    return df_data

In [ ]:
df_likes = manipular_e_filtrar_likes(df_original_likes)

# Adicionar interações do usuário

In [ ]:
extracao = ExtracaoInteracao(df_posts)
df_posts_interacoes = extracao.extract_interactions()

In [ ]:
df_posts_interacoes.head()

In [ ]:
df_posts_interacoes.to_csv('df_posts_interacoes.csv', index=False)

Função para calcular e visualizar a distribuição de postagens por usuário

In [ ]:
# Função para calcular e visualizar a distribuição de postagens por usuário
def distribuicao_posts(df, coluna_contagem, label_x, titulo):
    post_counts = df.groupby('id_usuario').size().reset_index(name=coluna_contagem)

    plt.hist(post_counts[coluna_contagem], bins=50)
    plt.xlabel(f'Número de {label_x}')
    plt.ylabel('Quantidade de Usuários')
    plt.title(titulo)
    plt.show()

    return post_counts

In [ ]:
quant_posts = distribuicao_posts(df_posts_interacoes, 'post_count', 'Postagens',
                                 'Distribuição de Postagens por Usuário')

In [ ]:
def distribuicao_postmessage(df, titulo):
    df_filtrado_msg = df[df['postMessage'].notnull() & df['postMessage'].str.strip() != '']

    post_counts_msg = df_filtrado_msg.groupby('id_usuario')['postMessage'].count().reset_index(name='quant_msg')

    plt.hist(post_counts_msg['quant_msg'], bins=50)
    plt.xlabel(f'Número de Postagens com Msg')
    plt.ylabel('Quantidade de Usuários')
    plt.title(titulo)
    plt.show()

    return post_counts_msg

In [ ]:
quant_msg = distribuicao_postmessage(df_posts_interacoes,
                                     'Distribuição de Postagens com Msg por Usuário')

In [ ]:
def distribuicao_poststory(df, titulo):
    df_filtrado_story = df[df['postStory'].notnull() & df['postStory'].str.strip() != '']

    post_counts_story = df_filtrado_story.groupby('id_usuario')['postStory'].count().reset_index(name='quant_story')

    # Visualizar a distribuição
    plt.hist(post_counts_story['quant_story'], bins=50)
    plt.xlabel(f'Número de Postagens com Story')
    plt.ylabel('Quantidade de Usuários')
    plt.title(titulo)
    plt.show()

    return post_counts_story

In [ ]:
quant_story = distribuicao_poststory(df_posts_interacoes, 'Distribuição de Postagens com Story por Usuário')

In [ ]:
def distribuicao_interacoes(df, coluna1, coluna2, coluna_contagem, label_x, titulo):
    if coluna2:
        interacoes = df.groupby('id_usuario')[[coluna1, coluna2]].sum().reset_index()
        interacoes[coluna_contagem] = interacoes[coluna1] + interacoes[coluna2]
    else:
        interacoes = df.groupby('id_usuario')[coluna1].sum().reset_index(name=coluna_contagem)

    plt.hist(interacoes[coluna_contagem], bins=50)
    plt.xlabel(label_x)
    plt.ylabel('Quantidade de Usuários')
    plt.title(titulo)
    plt.show()

    return interacoes

In [ ]:
quant_profile_cover = distribuicao_interacoes(df_posts_interacoes, 'quantProfile', 'quantCover',
                                              'quant_profile_cover', 'Quantidade de atualização de Perfil',
                                              'Distribuição de Atualização de Perfil por Usuário')

In [ ]:
# quantProfile,quantCover,quantAddPhotoWithOthers,quantIsWithOthers,quantAddPhoto,quantSharedPhoto,quantSharedVideo,quantSharedLink,quantSharedPost,quantSharedEvent,quantSharedMemory,quantStatus

In [ ]:
quant_profile = distribuicao_interacoes(df_posts_interacoes, 'quantProfile', None, 'quant_profile',
                                        'Quantidade de atualização de Perfil',
                                        'Distribuição de atualização de Perfil por Usuário')

In [ ]:
quant_cover = distribuicao_interacoes(df_posts_interacoes, 'quantCover', None, 'quant_cover',
                                      'Quantidade de atualização de Capa',
                                      'Distribuição de atualização de Capa por Usuário')

In [ ]:
quant_photo_with_others = distribuicao_interacoes(df_posts_interacoes, 'quantAddPhotoWithOthers', None,
                                                  'quant_photo_with_others', 'Quantidade de fotos com outras pessoas',
                                                  'Distribuição de fotos com outras pessoas por Usuário')

In [ ]:
quant_is_with_others = distribuicao_interacoes(df_posts_interacoes, 'quantIsWithOthers', None,
                                               'quant_is_with_others', 'Quantidade de fotos com outras pessoas',
                                               'Distribuição de fotos com outras pessoas por Usuário')

In [ ]:
quant_add_photo = distribuicao_interacoes(df_posts_interacoes, 'quantAddPhoto', None,
                                          'quant_add_photo', 'Quantidade de fotos adicionadas',
                                          'Distribuição de fotos adicionadas por Usuário')

In [ ]:
quant_shared_photo = distribuicao_interacoes(df_posts_interacoes, 'quantSharedPhoto', None,
                                             'quant_shared_photo', 'Quantidade de fotos compartilhadas',
                                             'Distribuição de fotos compartilhadas por Usuário')

In [ ]:
quant_shared_video = distribuicao_interacoes(df_posts_interacoes, 'quantSharedVideo', None,
                                             'quant_shared_video', 'Quantidade de vídeos compartilhados',
                                             'Distribuição de vídeos compartilhados por Usuário')

In [ ]:
quant_shared_link = distribuicao_interacoes(df_posts_interacoes, 'quantSharedLink', None,
                                            'quant_shared_link', 'Quantidade de links compartilhados',
                                            'Distribuição de links compartilhados por Usuário')

In [ ]:
quant_shared_post = distribuicao_interacoes(df_posts_interacoes, 'quantSharedPost', None,
                                            'quant_shared_post', 'Quantidade de posts compartilhados',
                                            'Distribuição de posts compartilhados por Usuário')

In [ ]:
quant_shared_event = distribuicao_interacoes(df_posts_interacoes, 'quantSharedEvent', None,
                                             'quant_shared_event', 'Quantidade de eventos compartilhados',
                                             'Distribuição de eventos compartilhados por Usuário')

In [ ]:
quant_shared_memory = distribuicao_interacoes(df_posts_interacoes, 'quantSharedMemory', None,
                                              'quant_shared_memory', 'Quantidade de memórias compartilhadas',
                                              'Distribuição de memórias compartilhadas por Usuário')

In [ ]:
quant_status = distribuicao_interacoes(df_posts_interacoes, 'quantStatus', None,
                                       'quant_status', 'Quantidade de status',
                                       'Distribuição de status por Usuário')

In [ ]:
def aplicar_kmeans(df, coluna_contagem, n_clusters=3, label_x='Postagens'):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    df['cluster'] = kmeans.fit_predict(df[[coluna_contagem]])

    sns.histplot(data=df, x=coluna_contagem, hue='cluster', bins=30, kde=True)
    plt.xlabel(f'Número de {label_x}')
    plt.ylabel('Frequência')
    plt.title(f'Distribuição dos Clusters de {label_x} por Usuário')
    plt.show()

    return kmeans, df

In [ ]:
def calcular_silhouette_e_inercia(df, kmeans, coluna_contagem):
    silhouette_avg = silhouette_score(df[[coluna_contagem]], kmeans.labels_)
    inercia = kmeans.inertia_

    print(f'Silhouette Score médio: {silhouette_avg}')
    print(f'Inércia do modelo: {inercia}')

In [ ]:
def calcular_inercia(df, coluna_contagem, max_clusters=10):
    inercia = []
    k = range(1, max_clusters + 1)
    for c in k:
        kmeans = KMeans(n_clusters=c, random_state=0)
        kmeans.fit(df[[coluna_contagem]])
        inercia.append(kmeans.inertia_)

    plt.plot(k, inercia, 'bx-')
    plt.xlabel('Número de Clusters')
    plt.ylabel('Inércia')
    plt.title('Método do Cotovelo para escolher o número ótimo de clusters')
    plt.show()

In [ ]:
def calcular_distancias_cluster(df, kmeans, coluna_contagem):
    distancias_intracluster = cdist(df[[coluna_contagem]], kmeans.cluster_centers_[kmeans.labels_], 'euclidean')
    media_intracluster = distancias_intracluster.mean()

    distancias_intercluster = cdist(kmeans.cluster_centers_, kmeans.cluster_centers_, 'euclidean')
    media_intercluster = distancias_intercluster[np.triu_indices_from(distancias_intercluster, 1)].mean()

    print(f'Média das distâncias intracluster: {media_intracluster}')
    print(f'Média das distâncias intercluster: {media_intercluster}')

In [ ]:
# Visualizar a distribuição
post_count = df_posts_interacoes.groupby('id_usuario').size().reset_index(name='quant_posts')

In [ ]:
# Aplicando a função
kmeans, post_count_original = aplicar_kmeans(post_count, 'quant_posts', 3)

In [ ]:
# Aplicando a função
calcular_silhouette_e_inercia(post_count, kmeans, 'quant_posts')

In [ ]:
# Aplicando a função
calcular_inercia(post_count, 'quant_posts')

In [ ]:
# Aplicando a função
calcular_distancias_cluster(post_count, kmeans, 'quant_posts')

In [ ]:
def decompor_e_teste_mk(df_cluster, periodo, tipo_serie='posts'):
    resultados_mk = []

    for usuario in df_cluster['id_usuario'].unique():
        df_usuario = df_cluster[df_cluster['id_usuario'] == usuario]

        eventos_por_dia = df_usuario.groupby('data').size()

        if len(eventos_por_dia) >= 2 * periodo:
            try:
                resultado = seasonal_decompose(eventos_por_dia, model='additive', period=periodo)
                tendencia = resultado.trend.dropna()

                if len(tendencia) > 0:
                    result = original_test(tendencia)

                    if hasattr(result, 'trend'):
                        resultados_mk.append({
                            'id_usuario': usuario,
                            'trend': result.trend,
                            'h': result.h,
                            'p-valor': result.p,
                            'z': result.z,
                            'Tau': result.Tau,
                            's': result.s,
                            'var_s': result.var_s,
                            'slope': result.slope,
                            'intercept': result.intercept
                        })
                    else:
                        print(f"Usuário {usuario} - Resultado do teste de Mann-Kendall sem 'trend'.")
                else:
                    print(f"Usuário {usuario} - Não foi possível decompor dados suficientes para tendência.")
            except ValueError as e:
                print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
        else:
            print(
                f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(eventos_por_dia)}")

    # Criando o DataFrame apenas se houver resultados suficientes
    if resultados_mk:
        df_resultados_mk = pd.DataFrame(resultados_mk)

        if 'trend' in df_resultados_mk.columns:
            contagem_tendencias = df_resultados_mk['trend'].value_counts()
            print(contagem_tendencias)

            total_usuarios = df_resultados_mk.shape[0]
            percentuais = (contagem_tendencias / total_usuarios) * 100
            print(percentuais)

            # Gráfico de barras das tendências
            plt.figure(figsize=(8, 6))
            sns.countplot(data=df_resultados_mk, x='trend', order=['increasing', 'decreasing', 'no trend'])
            plt.title(f'Distribuição de Tendências entre Usuários ({tipo_serie.capitalize()})')
            plt.xlabel('Tipo de Tendência')
            plt.ylabel('Número de Usuários')
            plt.show()

            # Gráfico de distribuição dos slopes
            plt.figure(figsize=(10, 6))
            sns.histplot(df_resultados_mk['slope'], bins=20, kde=True)
            plt.title(f'Distribuição dos Slopes entre Usuários ({tipo_serie.capitalize()})')
            plt.xlabel('Slope')
            plt.ylabel('Frequência')
            plt.show()

        else:
            print("Nenhuma tendência foi detectada nos dados.")

        return df_resultados_mk
    else:
        print("Nenhum dado disponível para análise.")
        return pd.DataFrame()

In [ ]:
# Criar um DataFrame para cada cluster de postagens
df_cluster0 = df_posts_interacoes[
    df_posts_interacoes['id_usuario'].isin(post_count[post_count['cluster'] == 0]['id_usuario'])]

df_cluster1 = df_posts_interacoes[
    df_posts_interacoes['id_usuario'].isin(post_count[post_count['cluster'] == 1]['id_usuario'])]

df_cluster2 = df_posts_interacoes[
    df_posts_interacoes['id_usuario'].isin(post_count[post_count['cluster'] == 2]['id_usuario'])]

In [ ]:
resultados_mk_original_cluster0 = decompor_e_teste_mk(df_cluster0, 7)
resultados_mk_original_cluster1 = decompor_e_teste_mk(df_cluster1, 7)
resultados_mk_original_cluster2 = decompor_e_teste_mk(df_cluster2, 7)

In [ ]:
resultados_mk_original_cluster0.to_csv('resultados_mk_original_cluster0.csv', index=False)
resultados_mk_original_cluster1.to_csv('resultados_mk_original_cluster1.csv', index=False)
resultados_mk_original_cluster2.to_csv('resultados_mk_original_cluster2.csv', index=False)

In [ ]:
df_cluster0.to_csv('df_cluster0.csv', index=False)
df_cluster1.to_csv('df_cluster1.csv', index=False)
df_cluster2.to_csv('df_cluster2.csv', index=False)

In [ ]:
def calcular_e_plotar_media_respostas_bdi(df_cluster, nome_cluster):
    # Calcular as respostas médias do BDI para cada usuário no cluster
    df_respostas_bdi = df_cluster.groupby(['id_usuario']).agg(
        pessimismo=('pessimismo', 'first'),
        tristeza=('tristeza', 'first'),
        fracasso=('fracasso', 'first'),
        prazer=('prazer', 'first'),
        culpa=('culpa', 'first'),
        punicao=('punicao', 'first'),
        estima=('estima', 'first'),
        critica=('critica', 'first'),
        suicida=('suicida', 'first'),
        choro=('choro', 'first'),
        agitacao=('agitacao', 'first'),
        interesse=('interesse', 'first'),
        indecisao=('indecisao', 'first'),
        desvalorizacao=('desvalorizacao', 'first'),
        energia=('energia', 'first'),
        sono=('sono', 'first'),
        irritabilidade=('irritabilidade', 'first'),
        apetite=('apetite', 'first'),
        concentracao=('concentracao', 'first'),
        fadiga=('fadiga', 'first'),
        int_sexo=('int_sexo', 'first'),
    ).reset_index()

    # Remover a coluna 'id_usuario'
    df_media_respostas_bdi = df_respostas_bdi.drop(columns=['id_usuario'])

    # Calcular a média de cada resposta do BDI
    media_respostas_bdi = df_media_respostas_bdi.mean()

    print(media_respostas_bdi)

    # Plotar as médias
    plt.figure(figsize=(10, 6))
    media_respostas_bdi.plot(kind='bar')
    plt.xlabel('Pergunta')
    plt.ylabel('Média da Resposta')
    plt.title(f'Média das Respostas do BDI - {nome_cluster}')
    plt.show()

    # Retornar as médias
    return media_respostas_bdi

In [ ]:
# Aplicando para o cluster 0
media_respostas_cluster0 = calcular_e_plotar_media_respostas_bdi(df_cluster0, "Cluster 0")

# Aplicando para o cluster 1
media_respostas_cluster1 = calcular_e_plotar_media_respostas_bdi(df_cluster1, "Cluster 1")

# Aplicando para o cluster 2
media_respostas_cluster2 = calcular_e_plotar_media_respostas_bdi(df_cluster2, "Cluster 2")

In [ ]:
def calcular_e_plotar_media_interacoes(df_cluster, nome_cluster):
    df_interacoes = df_cluster.groupby(['id_usuario']).agg(
        quant_profile=('quantProfile', 'sum'),
        quant_cover=('quantCover', 'sum'),
        quant_photo_with_others=('quantAddPhotoWithOthers', 'sum'),
        quant_is_with_others=('quantIsWithOthers', 'sum'),
        quant_add_photo=('quantAddPhoto', 'sum'),
        quant_shared_photo=('quantSharedPhoto', 'sum'),
        quant_shared_video=('quantSharedVideo', 'sum'),
        quant_shared_link=('quantSharedLink', 'sum'),
        quant_shared_post=('quantSharedPost', 'sum'),
        quant_shared_event=('quantSharedEvent', 'sum'),
        quant_shared_memory=('quantSharedMemory', 'sum'),
        quant_status=('quantStatus', 'sum'),
    ).reset_index()

    # Remover a coluna 'id_usuario'
    df_media_interacoes = df_interacoes.drop(columns=['id_usuario'])

    # Calcular a média de cada interação
    media_interacoes = df_media_interacoes.mean()

    print(media_interacoes)

    # Plotar as médias
    plt.figure(figsize=(10, 6))
    media_interacoes.plot(kind='bar')
    plt.xlabel('Tipo de Interação')
    plt.ylabel('Média da Interação')
    plt.title(f'Média das Interações - {nome_cluster}')
    plt.show()

    # Retornar as médias
    return media_interacoes

In [ ]:
# Aplicando para o cluster 0
media_interacoes_cluster0 = calcular_e_plotar_media_interacoes(df_cluster0, "Cluster 0")

# Aplicando para o cluster 1
media_interacoes_cluster1 = calcular_e_plotar_media_interacoes(df_cluster1, "Cluster 1")

# Aplicando para o cluster 2
media_interacoes_cluster2 = calcular_e_plotar_media_interacoes(df_cluster2, "Cluster 2")

# Likes

In [ ]:
df_likes_cluster0 = df_likes[df_likes['id_usuario'].isin(post_count[post_count['cluster'] == 0]['id_usuario'])]
df_likes_cluster1 = df_likes[df_likes['id_usuario'].isin(post_count[post_count['cluster'] == 1]['id_usuario'])]
df_likes_cluster2 = df_likes[df_likes['id_usuario'].isin(post_count[post_count['cluster'] == 2]['id_usuario'])]

In [ ]:
df_likes_cluster0.to_csv('df_likes_cluster0.csv', index=False)
df_likes_cluster1.to_csv('df_likes_cluster1.csv', index=False)
df_likes_cluster2.to_csv('df_likes_cluster2.csv', index=False)

In [ ]:
like_count_cluster0 = df_likes_cluster0.groupby('id_usuario').size().reset_index(name='quant_like_cluster0')
like_count_cluster1 = df_likes_cluster1.groupby('id_usuario').size().reset_index(name='quant_like_cluster1')
like_count_cluster2 = df_likes_cluster2.groupby('id_usuario').size().reset_index(name='quant_like_cluster2')

In [ ]:
resultados_mk_like_cluster0 = decompor_e_teste_mk(df_likes_cluster0, 7)
resultados_mk_like_cluster1 = decompor_e_teste_mk(df_likes_cluster1, 7)
resultados_mk_like_cluster2 = decompor_e_teste_mk(df_likes_cluster2, 7)